In [12]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import torch
import random
import numpy as np
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict
import seaborn as sns
from urllib.request import urlopen
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from sentence_transformers import SentenceTransformer
from time import gmtime, strftime
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
model_name = 'bert-base-nli-mean-tokens'
model_enc = SentenceTransformer(model_name)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english", truncation=True,max_seq_length=512)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [13]:
pwd

'C:\\Users\\bulat\\dataspell\\NIR'

In [12]:
### тестовая прогонка, поиск айдишника
data = []
s = 0

#filter_file = open('/content/drive/MyDrive/Books_filt_' + str(time.time()),'w')

good, bad ,id, s= 0,0,0,0
with gzip.open('Books.json.gz') as f:
    try:
        for k, l in enumerate(tqdm(f,total = 51311621)):
            try:
                a = json.loads(l.strip())
                try:
                    if 30 < len(a['reviewText']) < 512:
                        if id == '0001713353 ':
                            s += 1
                            #filter_file.write(json.dumps(a) + '\n')
                except KeyError:
                    pass
                except KeyboardInterrupt:
                    break
            except KeyboardInterrupt:
                break
    except KeyboardInterrupt:
        print(s)
print(s)




C:\Users\bulat\AppData\Local\Temp\ipykernel_2764\490725026.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for k, l in enumerate(tqdm(f,total = 51311621)):


  0%|          | 0/51311621 [00:00<?, ?it/s]

0


In [8]:
def filter_(data):
    """
    фильтрация предложения от знаков препинания
    """
    return data.replace("\n","").replace("\t","").replace(",","").replace(".","")

def negative_or_positive(sentence):
    """
    Проверка на ложно-положилтельные и ложно-отрицательные предложения
    """
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits

    predicted_class_id = logits.argmax().item()
    return model.config.id2label[predicted_class_id] == 'POSITIVE'

def make_embedings(sents):
    """
    Создание эмбедингов
    """
    return model_enc.encode(sents)

KeyboardInterrupt: 

In [10]:
bad_list = []
good_list = []
asin_list = []
same_asin_list = []

f = []
with gzip.open('/content/Books.json.gz') as file:
    for k, i in enumerate(file):
        seq = json.loads(i.strip())
        try:
            if 40 < len(seq['reviewText']) < 256:
                f.append(i)
                if k == 100000:
                    break
        except KeyError:
            ...


for k, l in enumerate(tqdm(f, total = 51311621)):
    same_asin_list = []
    good_list = []
    bad_list = []
    a = json.loads(l.strip())
    if a['asin'] in asin_list:
        continue
        print('continue on ', a['asin'])
    else:
        asin_list.append(a['asin'])
        for kk, ll in enumerate(f):
            b = json.loads(ll.strip())
            if b['asin'] == a['asin'] and b['reviewText'] != a['reviewText']:
                same_asin_list.append(b)
                print('append', b['asin'])
        #лист отзывов об одном и том же товаре собран, приступаем к разделению по оценке
        # и проверка тональности с помощью классификатора
        for i in same_asin_list:
            if int(i['overall']) <= 2:
                if negative_or_positive(i['reviewText'][:512]) == 0:
                    bad_list.append(filter_(i['reviewText']))
                    print(bad_list)
            if int(i['overall']) >= 4:
                if negative_or_positive(i['reviewText'][:512]) == 1:
                    good_list.append(filter_(i['reviewText']))
        if len(bad_list) != 0 and len(good_list) != 0:
            bad_list_vec = make_embedings(bad_list)
            good_list_vec = make_embedings(good_list)
            #Расчет косинусного расстояния и составление пар
            batch_size = 2
            batch_num = len(bad_list_vec) // batch_size
            sims_part = np.array
            for k in range( len(bad_list_vec) // batch_size ):
                start = k * batch_size
                stop =  (k + 1) * batch_size
                if (k+1) == batch_num:
                    stop =  len(bad_list_vec)
                print("start =",start, "stop =", stop)
                sims = cosine_similarity(bad_list_vec[start:stop], good_list_vec)
                sims_index = sims.argmax(axis = 1)
                with open("/content/drive/MyDrive/sims_2022_12.csv", 'a') as file_wr:
                    for i in range(len(sims)):
                        file_wr.write(bad_list[i+start] + ',' + good_list[sims_index[i]] + ',' + str(sims[i][sims_index[i]]) + '\n')


C:\Users\bulat\AppData\Local\Temp\ipykernel_2764\1879145896.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for k, l in enumerate(tqdm(f, total = 51311621)):


  0%|          | 0/51311621 [00:00<?, ?it/s]

append 0001713353
append 0001713353
append 0001713353
append 0001713353
append 0001713353
append 0001713353
append 0001713353
append 0001713353
append 0001713353
append 0001713353
append 0001713353
append 0001713353
append 0001713353
append 0001713353
append 0001713353
append 0001713353
append 0001713353
append 0001713353
append 0001713353
append 0001713353
append 0001713353
append 0001713353
append 0001713353
append 0001713353


NameError: name 'negative_or_positive' is not defined

In [ ]:
def divide_by_rating(data):
    for i in data:
        if i['overall'] in [1,2]:
            sent_train_bad.append(str(i.get('reviewText')).replace(',',''))
        elif i['overall'] in [5]:
            sent_train_good.append(str(i.get('reviewText')).replace(',',''))

    print('bad = ', len(sent_train_bad))
    print('good = ', len(sent_train_good))

In [ ]:
f = []
with gzip.open('/content/Books.json.gz') as file:
    for k, i in enumerate(file):
        f.append(i)
        if k == 100:
            break



In [ ]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import torch
import random
import numpy as np
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict
import seaborn as sns
from urllib.request import urlopen
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from sentence_transformers import SentenceTransformer
from time import gmtime, strftime
import concurrent.futures

################################################3
model_name = 'bert-base-nli-mean-tokens'
model_enc = SentenceTransformer(model_name)

from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english", truncation=True,max_seq_length=512)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

def filter_(data):
    return data.replace("\n","").replace("\t","").replace(",","").replace(".","")

def negative_or_positive(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits

    predicted_class_id = logits.argmax().item()
    return model.config.id2label[predicted_class_id] == 'POSITIVE'

def make_embedings(sents):
    return model_enc.encode(sents)

#####################

def get_pairs(asin, same_asin_list):
    bad_list = []
    good_list = []
    for i in same_asin_list:
        if int(i['overall']) <= 2:
            if negative_or_positive(i['reviewText'][:512]) == 0:
                bad_list.append(filter_(i['reviewText']))
        if int(i['overall']) >= 4:
            if negative_or_positive(i['reviewText'][:512]) == 1:
                good_list.append(filter_(i['reviewText']))
    if len(bad_list) != 0 and len(good_list) != 0:
        bad_list_vec = make_embedings(bad_list)
        good_list_vec = make_embedings(good_list)
        sims = cosine_similarity(bad_list_vec, good_list_vec)
        sims_index = sims.argmax(axis = 1)
        pairs = []
        for i in range(len(sims)):
            pairs.append((bad_list[i], good_list[sims_index[i]], sims[i][sims_index[i]]))
        return pairs
    else:
        return None

asin_list = set()
pairs_list = []
f = []
with gzip.open('/content/Books.json.gz') as file:
    for k, i in enumerate(file):
        seq = json.loads(i.strip())
        try:
            if 40 < len(seq['reviewText']) < 256:
                f.append(i)
                if k == 100000:
                    break
        except KeyError:
            ...
for k, l in enumerate(tqdm(f, total = 51311621)):
    same_asin_list = []
    a = json.loads(l.strip())
    if a['asin'] in asin_list:
        continue
    else:
        asin_list.add(a['asin'])
        for kk, ll